In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 19116312
paper_name = 'anand_payne_2009' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/hits.xlsx', sheet_name='Sheet1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 64 x 5


In [7]:
original_data.head()

,Single gene deletion strain,Gene product function,ALP defect,CPS defect,"Significance of ALP defect, p value"
0,AP-3,NaN,NaN,NaN,NaN
1,apl5Δ,δ Subunit of AP-3,3.1,−,<0.001
2,apl6Δ,β Subunit of AP-3,3.1,−,<0.001
3,apm3Δ,μ Subunit of AP-3,2.3,−,<0.001
4,aps3Δ,ς Subunit of AP-3,3.1,−,<0.001


In [14]:
original_data['gene'] = original_data['Single gene deletion strain'].apply(lambda x: x.split('(')[0][:-1])

In [15]:
# Eliminate all white spaces & capitalize
original_data['gene'] = clean_genename(original_data['gene'])

In [16]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['gene'], to='orf')

In [17]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                    Single gene deletion strain Gene product function  \
index_input                                                             
0                                          AP-3                   NaN   
5                             Vesicle formation                   NaN   
9                            Vacuolar proteases                   NaN   
12                                  HOPS/CORVET                   NaN   
20           Rabs and vesicle targeting factors                   NaN   
25                            SNAREs and fusion                   NaN   
29                         Vacuolar inheritance                   NaN   
32               Phosphatidylinositol synthesis                   NaN   
39                                     Retromer                   NaN   
45                            v-type H+-A TPase                   NaN   
59                                Miscellaneous                   NaN   

             ALP defect CPS defect Significance of

In [18]:
original_data = original_data.loc[t,:]

In [19]:
original_data['data'] = original_data['ALP defect'].astype(float)

In [20]:
original_data.set_index('orf', inplace=True)

In [21]:
original_data = original_data[['data']].copy()

In [22]:
original_data = original_data.groupby(original_data.index).mean()

In [23]:
original_data.shape

(53, 1)

# Prepare the final dataset

In [24]:
data = original_data.copy()

In [25]:
dataset_ids = [122]
datasets = datasets.reindex(index=dataset_ids)

In [26]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [27]:
data.head()

dataset_id,122
data_type,value
orf,
YAL002W,2.3
YAL026C,3.2
YBR011C,2.0
YBR097W,5.0
YBR127C,1.7


## Subset to the genes currently in SGD

In [28]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [29]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,122
,data_type,value
gene_id,orf,
2,YAL002W,2.3
24,YAL026C,3.2
207,YBR011C,2.0
292,YBR097W,5.0
321,YBR127C,1.7


# Normalize

In [30]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [31]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [32]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        122           
data_type       value     valuez
gene_id orf                     
2       YAL002W   2.3   5.081919
24      YAL026C   3.2   7.070496
207     YBR011C   2.0   4.419060
292     YBR097W   5.0  11.047650
321     YBR127C   1.7   3.756201

# Print out

In [33]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [34]:
from IO.save_data_to_db3 import *

In [35]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 19116312...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  5.76it/s]


Updating the data_modified_on field...
